In [1]:
import pandas as pd
import numpy as np

In [7]:
movies_df = pd.read_csv("data/movies.csv")
movie_genres_df = pd.read_csv("data/movie_genres.csv")
ratings_df = pd.read_csv("data/ratings.csv")
movie_keyboards_df = pd.read_csv("data/movies_keywords.csv")

In [3]:
movies_df.head()

,Unnamed: 0,movieId,title,genres
0,0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,2,Jumanji (1995),Adventure|Children|Fantasy
2,2,3,Grumpier Old Men (1995),Comedy|Romance
3,3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,4,5,Father of the Bride Part II (1995),Comedy


In [9]:
ratings_df.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
movie_genres_df.head()

,movie_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [8]:
movie_keyboards_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,keywords
0,tt0111161,movie,The Shawshank Redemption,The Shawshank Redemption,0,1994,\N,142,Drama,9.3,2057323,"wrongful-imprisonment,escape-from-prison,based..."
1,tt0468569,movie,The Dark Knight,The Dark Knight,0,2008,\N,152,"Action,Crime,Drama",9.0,2023734,"dc-comics,moral-dilemma,psychopath,clown,scarr..."
2,tt1375666,movie,Inception,Inception,0,2010,\N,148,"Action,Adventure,Sci-Fi",8.8,1802989,"dream,subconscious,ambiguous-ending,thief,psyc..."
3,tt0137523,movie,Fight Club,Fight Club,0,1999,\N,139,Drama,8.8,1646400,"surprise-ending,fighting,multiple-personality-..."
4,tt0110912,movie,Pulp Fiction,Pulp Fiction,0,1994,\N,154,"Crime,Drama",8.9,1607874,"nonlinear-timeline,black-comedy,overdose,bonda..."


In [9]:
print(movie_genres_df.shape)
print(movie_keyboards_df.shape)
print(movies_df.shape)
print(ratings_df.shape)

(1682, 24)
(20, 12)
(2000, 4)
(2000, 4)


In [11]:
import nltk
import gensim

## Content Based Approach
Finding similarity between movies

In [13]:
movie_genres_df.drop(columns=['release date','video release date','IMDb URL'], inplace=True)

In [14]:
movie_genres_df.head()

,movie_id,movie title,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [15]:
movie_genres_df.columns

Index(['movie_id', 'movie title', 'unknown', 'Action', 'Adventure',
       'Animation', 'Children's', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western'],
      dtype='object')

In [17]:
updated_movie_genres_df = movie_genres_df.drop(columns=['movie_id','movie title'])

In [18]:
updated_movie_genres_df.head()

,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [19]:
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
index = cosine_similarity(updated_movie_genres_df)

In [22]:
index

array([[1.        , 0.        , 0.        , ..., 0.        , 0.57735027,
        0.        ],
       [0.        , 1.        , 0.57735027, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.57735027, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.70710678],
       [0.57735027, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.70710678, 0.        ,
        1.        ]])

In [78]:
def get_similarity_movie(movie_id):
    specific_genres = movie_genres_df.loc[movie_genres_df["movie_id"] == movie_id].values.tolist()[0][2:]
    query_movie_name = movie_genres_df.loc[movie_genres_df["movie_id"] == movie_id].values.tolist()[0][1]
    print("getting similarity for " + query_movie_name + ": \n")
    similarities = []
    for movie in range(0,updated_movie_genres_df.shape[0]):
        sim = cosine_similarity([updated_movie_genres_df.loc[movie]],[specific_genres])
        movie_name = movie_genres_df.iloc[movie]['movie title']
        similarities.append((movie_name, sim.tolist()[0][0]))
    similarities.sort(key= lambda x:x[1], reverse = True)
    print(similarities[:10])

In [103]:
get_similarity_movie(4)

getting similarity for Get Shorty (1995): 

[('Get Shorty (1995)', 1.0000000000000002), ('Faster Pussycat! Kill! Kill! (1965)', 1.0000000000000002), ('Best Men (1997)', 0.8660254037844388), ('Doom Generation, The (1995)', 0.816496580927726), ('Eat Drink Man Woman (1994)', 0.816496580927726), ('Ed Wood (1994)', 0.816496580927726), ("What's Eating Gilbert Grape (1993)", 0.816496580927726), ('Welcome to the Dollhouse (1995)', 0.816496580927726), ('Swingers (1996)', 0.816496580927726), ('Citizen Ruth (1996)', 0.816496580927726)]


In [81]:
get_similarity_movie(56)

getting similarity for Pulp Fiction (1994): 

[('Pulp Fiction (1994)', 0.9999999999999998), ("Carlito's Way (1993)", 0.9999999999999998), ('GoodFellas (1990)', 0.9999999999999998), ('Donnie Brasco (1997)', 0.9999999999999998), ('Jackie Brown (1997)', 0.9999999999999998), ('Bonnie and Clyde (1967)', 0.9999999999999998), ('Sleepers (1996)', 0.9999999999999998), ('Twilight (1998)', 0.9999999999999998), ('Newton Boys, The (1998)', 0.9999999999999998), ('They Made Me a Criminal (1939)', 0.9999999999999998)]


In [83]:
get_similarity_movie(842)

getting similarity for Pollyanna (1960): 

[('Babe (1995)', 1.0000000000000002), ('Pollyanna (1960)', 1.0000000000000002), ('Doom Generation, The (1995)', 0.816496580927726), ('Eat Drink Man Woman (1994)', 0.816496580927726), ('Ed Wood (1994)', 0.816496580927726), ('Santa Clause, The (1994)', 0.816496580927726), ("What's Eating Gilbert Grape (1993)", 0.816496580927726), ('Welcome to the Dollhouse (1995)', 0.816496580927726), ('Home Alone (1990)', 0.816496580927726), ('D3: The Mighty Ducks (1996)', 0.816496580927726)]


In [84]:
from sklearn.metrics.pairwise import pairwise_distances 
movie_similarity = 1- pairwise_distances(updated_movie_genres_df, metric='cosine')

In [85]:
movie_similarity

array([[1.        , 0.        , 0.        , ..., 0.        , 0.57735027,
        0.        ],
       [0.        , 1.        , 0.57735027, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.57735027, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.70710678],
       [0.57735027, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.70710678, 0.        ,
        1.        ]])

In [104]:
def get_similarity_2_movies(movie_id):
    movie_ids = movie_genres_df['movie_id'].values.tolist()
    movie_titles = movie_genres_df['movie title'].values.tolist()
    similarity = movie_similarity[movie_id].tolist()
    data = {'movie_id':movie_ids,'movie title':movie_titles,'similarity':similarity}
    sim_df = pd.DataFrame(data,columns=['movie_id','movie title','similarity'])
    return sim_df

In [108]:
get_similarity_2_movies(3).sort_values(by='similarity', ascending=False)[:20]

,movie_id,movie title,similarity
3,4,Get Shorty (1995),1.000000
73,74,Faster Pussycat! Kill! Kill! (1965),1.000000
1137,1138,Best Men (1997),0.866025
344,345,Deconstructing Harry (1997),0.816497
1596,1597,Romper Stomper (1992),0.816497
1402,1403,Caro Diario (Dear Diary) (1994),0.816497
952,953,Unstrung Heroes (1995),0.816497
455,456,Beverly Hills Ninja (1997),0.816497
963,964,"Month by the Lake, A (1995)",0.816497
1587,1588,Salut cousin! (1996),0.816497


## Collaborating Filtering
### 2.1 Memory Based User Based 

In [8]:
ratings_df.nunique()

user_id             943
movie_id           1682
rating                5
unix_timestamp    49282
dtype: int64

In [10]:
ratings_df.shape

(100000, 4)

In [11]:
ratings_df.drop(columns=['unix_timestamp'], inplace=True)

In [12]:
ratings_df.head()

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


### Split the data into train and test

In [14]:
from sklearn.model_selection import train_test_split

In [16]:
train, test = train_test_split(ratings_df,train_size=0.8, random_state=45)

In [18]:
train.shape

(80000, 3)

In [19]:
test.shape

(20000, 3)

In [20]:
n_users = ratings_df.user_id.nunique()
n_movies = ratings_df.movie_id.nunique()

In [21]:
print(n_users)
print(n_movies)

943
1682


### Creating a user * movie data matrix

In [23]:
data_matrix = np.zeros((n_users,n_movies))

In [36]:
data_matrix.shape

(943, 1682)

### Now we have the matrix we fill the ratings

In [37]:
for line in train.itertuples():
    data_matrix[line[1]-1,line[2]-1] = line[3]

In [38]:
data_matrix

array([[5., 3., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

### Now we create data matrix for test data

In [39]:
data_matrix_test = np.zeros((n_users,n_movies))

In [40]:
for line in test.itertuples():
    data_matrix_test[line[1]-1,line[2]-1] = line[3]

In [41]:
data_matrix_test.shape

(943, 1682)

In [42]:
data_matrix_test

array([[0., 0., 4., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Now we create data_matrix for full data

In [43]:
data_matrix_full = np.zeros((n_users,n_movies))

In [44]:
for line in ratings_df.itertuples():
    data_matrix_full[line[1]-1,line[2]-1] = line[3]

In [48]:
from sklearn.metrics.pairwise import pairwise_distances

In [49]:
user_similarity = 1- pairwise_distances(data_matrix, metric="cosine")

In [50]:
user_similarity.shape

(943, 943)

In [51]:
item_similarities = 1 - pairwise_distances(data_matrix.T, metric="cosine")

In [52]:
item_similarities.shape

(1682, 1682)

In [54]:
item_prediction = np.dot(user_similarity, data_matrix_test)

In [55]:
item_prediction

array([[75.09272964, 26.03239329, 17.36487041, ...,  0.        ,
         0.        ,  0.        ],
       [51.69127647,  9.38207703,  9.18278632, ...,  0.        ,
         0.        ,  0.        ],
       [17.98809138,  4.26065349,  4.54252118, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [43.18301137,  8.63719012,  8.67914653, ...,  0.        ,
         0.        ,  0.        ],
       [50.48566855, 15.70543162,  7.7723109 , ...,  0.        ,
         0.        ,  0.        ],
       [70.5084916 , 25.84137081, 13.70035595, ...,  0.        ,
         0.        ,  0.        ]])

In [56]:
item_prediction.shape

(943, 1682)

In [57]:
prediction_df = pd.DataFrame(item_prediction)

In [58]:
prediction_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,75.092730,26.032393,17.364870,62.673237,16.342949,3.162578,64.184943,45.222422,59.618847,19.953852,...,0.0,0.0,0.616669,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,51.691276,9.382077,9.182786,27.901109,7.983499,2.724819,48.141782,25.124366,45.106714,12.892081,...,0.0,0.0,0.664267,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17.988091,4.260653,4.542521,11.680759,2.921409,1.075073,17.260724,10.099117,16.794345,3.705296,...,0.0,0.0,0.505283,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,26.652907,6.020866,5.134496,14.034047,3.531117,1.055313,22.233322,10.515605,18.202196,4.369093,...,0.0,0.0,0.107254,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,64.366961,21.678807,11.072046,47.889828,13.107265,1.868597,49.394095,35.689413,41.636565,12.110950,...,0.0,0.0,0.364056,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
recommended_movie_df = pd.DataFrame(prediction_df.iloc[117].sort_values(ascending=False))
recommended_movie_df

,117
49,77.908156
99,67.750878
180,65.518940
173,59.645050
55,56.731490
...,...
1401,0.000000
1399,0.000000
1398,0.000000
1397,0.000000


In [67]:
recommended_movie_df.reset_index(inplace=True)

ValueError: cannot insert level_0, already exists

In [70]:
recommended_movie_df.drop(columns=['level_0'], inplace=True)

In [71]:
recommended_movie_df.head()

,index,117
0,49,77.908156
1,99,67.750878
2,180,65.518940
3,173,59.645050
4,55,56.731490


In [72]:
recommended_movie_df.columns = ['movie_id','prediction']

In [73]:
recommended_movie_df.head()

,movie_id,prediction
0,49,77.908156
1,99,67.750878
2,180,65.518940
3,173,59.645050
4,55,56.731490


In [80]:
recommended_movie_df

,movie_id,prediction
0,49,77.908156
1,99,67.750878
2,180,65.518940
3,173,59.645050
4,55,56.731490
...,...,...
1677,1401,0.000000
1678,1399,0.000000
1679,1398,0.000000
1680,1397,0.000000


In [77]:
movie_genres_df.shape

(1682, 24)

In [81]:
merged = pd.merge(recommended_movie_df, movie_genres_df, on='movie_id', how='left')

In [82]:
merged

,movie_id,prediction,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,49,77.908156,I.Q. (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?I.Q.%20(1994),0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,99,67.750878,Snow White and the Seven Dwarfs (1937),01-Jan-1937,NaN,http://us.imdb.com/M/title-exact?Snow%20White%...,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,180,65.518940,Apocalypse Now (1979),01-Jan-1979,NaN,http://us.imdb.com/M/title-exact?Apocalypse%20...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,173,59.645050,"Princess Bride, The (1987)",01-Jan-1987,NaN,http://us.imdb.com/M/title-exact?Princess%20Br...,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,55,56.731490,"Professional, The (1994)",01-Jan-1994,NaN,http://us.imdb.com/Title?L%E9on+(1994),0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1401,0.000000,M. Butterfly (1993),01-Jan-1993,NaN,http://us.imdb.com/M/title-exact?M.%20Butterfl...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1678,1399,0.000000,Stranger in the House (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?imdb-title-12...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1679,1398,0.000000,Anna (1996),13-Nov-1996,NaN,http://us.imdb.com/M/title-exact?Anna%20(1996),0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1680,1397,0.000000,Of Human Bondage (1934),01-Jan-1934,NaN,http://us.imdb.com/M/title-exact?Of%20Human%20...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
merged['movie title'].head(10)

0                               I.Q. (1994)
1    Snow White and the Seven Dwarfs (1937)
2                     Apocalypse Now (1979)
3                Princess Bride, The (1987)
4                  Professional, The (1994)
5                         Sting, The (1973)
6                 Dances with Wolves (1990)
7                         Unforgiven (1992)
8                            Aladdin (1992)
9                      Seven (Se7en) (1995)
Name: movie title, dtype: object